In [201]:
import pandas as pd
import numpy as np
import re
import collections
import time
import recordlinkage
import jellyfish

In [188]:
openhouse = pd.read_csv('open-houses-montreal.csv',encoding='utf-8')
openhouse = openhouse[~openhouse.muncipality_id.isna()]

house_original = openhouse.copy()

In [189]:
openhouse = openhouse[['muncipality_id','category','listing_price','sales_type','address','longitude',
                       'latitude','year_built','retrieved_at']]

In [190]:
openhouse.muncipality_id = openhouse.muncipality_id.astype('int')

In [191]:
openhouse.address = openhouse.address.apply(lambda x: [i.rstrip(' ') for i in x.split(', ')])

In [192]:
muncipality_name = dict()
for muncipality_id in np.unique(openhouse.muncipality_id):
    
    if len(openhouse[openhouse.muncipality_id==muncipality_id]) > 20:
    
        L = [j for i in openhouse[openhouse.muncipality_id==muncipality_id].address for j in i]

        c = collections.Counter(L)

        muncipality_name [muncipality_id] = c.most_common()[0][0]

In [7]:
print len(openhouse)
neighbourhood = [np.nan] * len(openhouse)
start = time.time()
for i in range(len(openhouse)):
    #if np.mod(i,1000)==0:
    #    print i,time.time()-start
        
    try:
        openhouse.iloc[i,4].remove(muncipality_name[openhouse.muncipality_id.iloc[i]])
    except:
        pass
    
    for j in openhouse.iloc[i,4]:
        if 'Neighbourhood' in j:
            neighbourhood[i] = j
            openhouse.iloc[i,4].remove(j)

34905


In [8]:
np.unique(neighbourhood)

array(['Neighbourhood Ahuntsic Central', 'Neighbourhood Ahuntsic East',
       'Neighbourhood Ahuntsic West',
       'Neighbourhood Beacon Hill (North East)',
       'Neighbourhood Beaurepaire (South West)',
       'Neighbourhood Bois de Saraguay',
       'Neighbourhood Carrefour Saint-Martin',
       'Neighbourhood Cartierville', 'Neighbourhood Central',
       'Neighbourhood Central East', 'Neighbourhood Central South',
       'Neighbourhood Central West', 'Neighbourhood City',
       'Neighbourhood Cit\xc3\xa9 du Havre',
       'Neighbourhood Cit\xc3\xa9-Jardin',
       'Neighbourhood Cit\xc3\xa9-Universitaire',
       'Neighbourhood C\xc3\xb4te-des-Neiges', 'Neighbourhood East',
       'Neighbourhood Forest Gardens (Central North)',
       'Neighbourhood Golden Square Mile', 'Neighbourhood Griffintown',
       'Neighbourhood Hochelaga-Maisonneuve',
       "Neighbourhood L'\xc3\x8ele-Bizard",
       'Neighbourhood La Cit\xc3\xa9 du Multim\xc3\xa9dia',
       'Neighbourhood La Petite

In [9]:
openhouse['neighbourhood'] = neighbourhood

In [10]:
len_address = openhouse.address.apply(lambda x: len(x))
collections.Counter(len_address)

Counter({1: 159, 2: 18693, 3: 16032, 4: 19, 5: 2})

# address lenth >3 are mostly un-popular munipality

In [11]:
for i in openhouse.address[len_address>3].index:
    print openhouse.loc[i].muncipality_id 
    print openhouse.loc[i].address

279
['2663', 'Chemin Sainte-Foy', 'apt. 108', 'Sainte-Foy/Sillery/Cap-Rouge (Qu\xc3\xa9bec)']
115
['1013', 'Rue du Parc', 'Rimouski', 'Quartier Pointe-au-P\xc3\xa8re']
276
['1195', 'Avenue Royale', 'Beauport (Qu\xc3\xa9bec)', 'Quartier Vieux-Bourg']
1264
['2021', 'Rue Saint-Jean-Baptiste', 'Jonqui\xc3\xa8re (Saguenay)', 'Quartier Jonqui\xc3\xa8re']
804
['587', 'Rue Guillemette', 'app. 86', 'Fabreville (Laval)', 'Quartier Est']
281
['1516 - 1518', "Rue de l'Esplanade", 'La Haute-Saint-Charles (Qu\xc3\xa9bec)', 'Quartier Val-B\xc3\xa9lair']
1133
['1', 'Chemin du Lac-Gueguen', "Val-d'Or", 'Quartier Ext\xc3\xa9rieur Est']
1085
['358A - 360', 'Rue Pauly', 'Rouyn-Noranda', 'Quartier Noranda-Nord']
463
['183 - 185', 'Avenue Godefroy', 'B\xc3\xa9cancour', 'Quartier Saint-Gr\xc3\xa9goire']
843
['207', 'Rue de la Commune Ouest', 'apt. RDC', '2,3']
276
['127', 'Rue Latouche', 'Beauport (Qu\xc3\xa9bec)', 'Quartier Chutes-Montmorency']
1013
['994', 'boulevard Maloney Est', 'Gatineau (Gatineau)', 'Q

# address lenth == 1 are mostly lot, land, business or without listing price

In [12]:
short_add = openhouse[len_address==1]

In [13]:
short_add[(short_add.category!='Lot') & (short_add.category!='Land') & (short_add.listing_price>1)]

,muncipality_id,category,listing_price,sales_type,address,longitude,latitude,year_built,retrieved_at,neighbourhood
4745,842,Business,155000.0,sale,[Rue Saint-Denis],-73.569246,45.517520,NaN,2018-06-07,Neighbourhood Le Plateau-Mont-Royal
25781,844,Business,150000.0,sale,[Rue Beaubien Est],-73.588341,45.552852,NaN,2018-01-23,Neighbourhood Rosemont North
25932,846,Income properties,1550000.0,sale,[Rue Notre-Dame Est],-73.496554,45.683795,1958.0,2018-01-23,Neighbourhood Pointe-aux-Trembles
33420,838,Business,2400000.0,sale,[Rue Dalou],-73.625196,45.482885,NaN,2018-01-22,Neighbourhood Côte-des-Neiges
33533,840,Business,750000.0,sale,[boulevard Gouin Ouest],-73.723304,45.529636,NaN,2018-01-22,Neighbourhood Cartierville
34103,817,Business,365000.0,sale,[boulevard Henri-Bourassa Est],-73.648396,45.587735,NaN,2018-01-22,NaN


# address lenth == 3 and without apt. or suite are mostly un-popular munipality

In [15]:
openhouse[(len_address==3)][openhouse.address[(len_address==3)].apply(lambda x: len(set(['apt.','suite'])
                                                               .intersection(set(x[2].split(' '))))==0)]

,muncipality_id,category,listing_price,sales_type,address,longitude,latitude,year_built,retrieved_at,neighbourhood
2156,289,House,NaN,sale,"[254, Rue de l'Acadie, Les Chutes-de-la-Chaudi...",-71.311030,46.707160,1978.0,2018-07-03,Neighbourhood Saint-Nicolas
2158,118,House,NaN,sale,"[8, 6e Avenue, Saint-Fabien]",-68.868167,48.293496,0.0,2018-07-03,NaN
2159,593,Lot,NaN,sale,"[33, Carré George-Adams, Bromont]",-72.782772,45.276457,NaN,2018-07-03,NaN
2160,747,House,NaN,sale,"[3775, Rue La Durantaye, Saint-Hubert (Longueu...",-73.358086,45.454009,2009.0,2018-07-03,Neighbourhood Le Boisé de Saint-Hubert
2161,799,House,NaN,sale,"[2791, Rue Chauvette, Mascouche]",-73.606795,45.754472,1975.0,2018-07-03,Neighbourhood City
2162,281,House,NaN,sale,"[1000, Rue du Cabestan, La Haute-Saint-Charles...",-71.426416,46.845477,2009.0,2018-07-03,Neighbourhood Val-Bélair
2163,928,House,NaN,sale,"[2129, Terrasse Jourdain, Sainte-Sophie]",-73.890200,45.821220,1978.0,2018-07-03,NaN
2164,948,House,NaN,sale,"[2105, Chemin de la Baie-Noire, Wentworth-Nord]",-74.442791,45.760502,1954.0,2018-07-03,NaN
2165,907,House,NaN,sale,"[1240, Montée Noire, Sainte-Justine-de-Newton]",-74.399402,45.379873,2010.0,2018-07-03,NaN
2166,673,House,NaN,sale,"[4400, Chemin du Lac, Saint-Gabriel-de-Brandon]",-73.340222,46.262654,0.0,2018-07-03,NaN


# process address lenth==3 records with apt. or suite. no.

In [131]:
openhouse_3 = openhouse[(len_address==3)][openhouse.address[(len_address==3)].apply(lambda x: len(set(['apt.','suite'])
                                                               .intersection(set(x[2].split(' '))))>0)]

In [132]:
openhouse_3['apt'] = openhouse_3.address.apply(lambda x: x[2].replace('apt.','').replace('suite','').replace(' ',''))

In [133]:
openhouse_3['No.'] = openhouse_3.address.apply(lambda x: x[0])

openhouse_3['street'] = openhouse_3.address.apply(lambda x: x[1])

In [134]:
openhouse_3

,muncipality_id,category,listing_price,sales_type,address,longitude,latitude,year_built,retrieved_at,neighbourhood,apt,No.,street
1,815,Condo,174900.0,sale,"[6801, boulevard des Roseraies, apt. 303]",-73.555285,45.597026,1990.0,2018-07-25,NaN,303,6801,boulevard des Roseraies
2,843,Condo,749000.0,sale,"[1210, boulevard De Maisonneuve Ouest, apt. 21A]",-73.575657,45.499809,2006.0,2018-07-25,Neighbourhood Golden Square Mile,21A,1210,boulevard De Maisonneuve Ouest
3,836,Condo,289000.0,sale,"[120, Place Donnacona, apt. 302]",-73.809583,45.486850,2004.0,2018-07-25,Neighbourhood Central,302,120,Place Donnacona
7,845,Condo,189000.0,sale,"[2560, Avenue Bennett, apt. 3]",-73.547969,45.557645,1992.0,2018-07-25,Neighbourhood Hochelaga-Maisonneuve,3,2560,Avenue Bennett
10,822,Condo,350000.0,sale,"[5740, Avenue Rembrandt, apt. 602]",-73.659603,45.476015,1980.0,2018-07-25,NaN,602,5740,Avenue Rembrandt
13,827,Condo,339000.0,sale,"[3175, Avenue Ernest-Hemingway, apt. 307]",-73.719273,45.506360,2007.0,2018-07-25,Neighbourhood New Saint-Laurent/Bois-Franc,307,3175,Avenue Ernest-Hemingway
14,843,Condo,1325000.0,sale,"[1455, Rue Sherbrooke Ouest, apt. 1601]",-73.580818,45.497798,1966.0,2018-07-25,Neighbourhood Golden Square Mile,1601,1455,Rue Sherbrooke Ouest
16,840,Condo,279000.0,sale,"[1475, Place de Louvain, apt. 6]",-73.643332,45.562717,1986.0,2018-07-25,Neighbourhood Ahuntsic West,6,1475,Place de Louvain
24,815,Condo,229500.0,sale,"[7200, Avenue M-B-Jodoin, apt. 807]",-73.583221,45.606265,1987.0,2018-07-25,NaN,807,7200,Avenue M-B-Jodoin
25,843,Condo,1100000.0,sale,"[801, Rue de la Commune Est, apt. 804]",-73.549404,45.512143,2007.0,2018-07-25,Neighbourhood Old Montréal,804,801,Rue de la Commune Est


In [135]:
start = time.time()

pc = recordlinkage.BlockIndex(on=['muncipality_id','category','sales_type'])

pairs = pc.index(openhouse_3)

print 'Index takes:',time.time()-start

Index takes: 9.71868395805


In [136]:
len(pairs)

7356551

In [120]:
start = time.time()

pc1 = recordlinkage.BlockIndex(on=['muncipality_id'])

pairs = pc1.index(openhouse_3)

print 'Index takes:',time.time()-start

print len(pairs)

Index takes: 16.842028141
16202869


In [121]:
start = time.time()

pc2 = recordlinkage.BlockIndex(on=['muncipality_id','category'])

pairs = pc2.index(openhouse_3)

print 'Index takes:',time.time()-start

print len(pairs)

Index takes: 11.3599848747
7371599


In [137]:
compare_house = recordlinkage.Compare()

compare_house.string('street','street', method='jarowinkler', threshold=0.95, label='street')

compare_house.exact('apt','apt',label='apt')

compare_house.exact('No.','No.',label='No.')

<Compare>

In [138]:
openhouse_3.to_csv('openhouse_3.csv',index=False,encoding='utf-8')
openhouse_3 = pd.read_csv('openhouse_3.csv',encoding='utf-8')

In [139]:
openhouse_3

,muncipality_id,category,listing_price,sales_type,address,longitude,latitude,year_built,retrieved_at,neighbourhood,apt,No.,street
0,815,Condo,174900.0,sale,"[6801, boulevard des Roseraies, apt. 303]",-73.555285,45.597026,1990.0,2018-07-25,NaN,303,6801,boulevard des Roseraies
1,843,Condo,749000.0,sale,"[1210, boulevard De Maisonneuve Ouest, apt. 21A]",-73.575657,45.499809,2006.0,2018-07-25,Neighbourhood Golden Square Mile,21A,1210,boulevard De Maisonneuve Ouest
2,836,Condo,289000.0,sale,"[120, Place Donnacona, apt. 302]",-73.809583,45.486850,2004.0,2018-07-25,Neighbourhood Central,302,120,Place Donnacona
3,845,Condo,189000.0,sale,"[2560, Avenue Bennett, apt. 3]",-73.547969,45.557645,1992.0,2018-07-25,Neighbourhood Hochelaga-Maisonneuve,3,2560,Avenue Bennett
4,822,Condo,350000.0,sale,"[5740, Avenue Rembrandt, apt. 602]",-73.659603,45.476015,1980.0,2018-07-25,NaN,602,5740,Avenue Rembrandt
5,827,Condo,339000.0,sale,"[3175, Avenue Ernest-Hemingway, apt. 307]",-73.719273,45.506360,2007.0,2018-07-25,Neighbourhood New Saint-Laurent/Bois-Franc,307,3175,Avenue Ernest-Hemingway
6,843,Condo,1325000.0,sale,"[1455, Rue Sherbrooke Ouest, apt. 1601]",-73.580818,45.497798,1966.0,2018-07-25,Neighbourhood Golden Square Mile,1601,1455,Rue Sherbrooke Ouest
7,840,Condo,279000.0,sale,"[1475, Place de Louvain, apt. 6]",-73.643332,45.562717,1986.0,2018-07-25,Neighbourhood Ahuntsic West,6,1475,Place de Louvain
8,815,Condo,229500.0,sale,"[7200, Avenue M-B-Jodoin, apt. 807]",-73.583221,45.606265,1987.0,2018-07-25,NaN,807,7200,Avenue M-B-Jodoin
9,843,Condo,1100000.0,sale,"[801, Rue de la Commune Est, apt. 804]",-73.549404,45.512143,2007.0,2018-07-25,Neighbourhood Old Montréal,804,801,Rue de la Commune Est


In [140]:
start = time.time()

features = compare_house.compute(pairs, openhouse_3, openhouse_3)

print 'Take',time.time()-start

Take 95.2610638142


In [141]:
indexs = features[features.sum(axis=1)>2]

In [142]:
indexs

,,street,apt,No.
362,5766,1.0,1,1
2287,14593,1.0,1,1
7652,8373,1.0,1,1
917,15794,1.0,1,1
221,11832,1.0,1,1
7872,14962,1.0,1,1
4400,4974,1.0,1,1
13039,15420,1.0,1,1
224,6207,1.0,1,1
473,7345,1.0,1,1


In [143]:
i = 4400
openhouse_3.loc[[i]+list(indexs.loc[i].index)]

,muncipality_id,category,listing_price,sales_type,address,longitude,latitude,year_built,retrieved_at,neighbourhood,apt,No.,street
4400,843,Condo,625000.0,sale,"[1000, Rue de la Commune Est, apt. 518]",-73.549257,45.512909,2004.0,2018-04-23,Neighbourhood Old Montréal,518,1000,Rue de la Commune Est
4974,843,Condo / Apartment,0.0,rent,"[1000, Rue de la Commune Est, apt. 518]",-73.549257,45.512909,2004.0,2018-04-14,Neighbourhood Old Montréal,518,1000,Rue de la Commune Est


# process address lenth==2 records

In [144]:
openhouse_2 = openhouse[(len_address==2)]

In [147]:
openhouse_2['street'] = openhouse_2.address.apply(lambda x:x[1])

openhouse_2['No.'] = openhouse_2.address.apply(lambda x:x[0])

/Users/lingao/.virtualenvs/scratch/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/lingao/.virtualenvs/scratch/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [153]:
openhouse_2.to_csv('openhouse_2.csv',index=False,encoding='utf-8')
openhouse_2 = pd.read_csv('openhouse_2.csv',encoding='utf-8')

In [149]:
start = time.time()

pc3 = recordlinkage.BlockIndex(on=['muncipality_id','category','sales_type'])

pairs = pc3.index(openhouse_2)

print 'Index takes:',time.time()-start

print len(pairs)

Index takes: 2.46398806572
1892300


In [151]:
compare_house = recordlinkage.Compare()

compare_house.string('street','street', method='jarowinkler', threshold=0.95, label='street')

compare_house.exact('No.','No.',label='No.')

<Compare>

In [154]:
start = time.time()

features = compare_house.compute(pairs, openhouse_2, openhouse_2)

print 'Take',time.time()-start

Take 10.1757278442


In [156]:
indexs = features[features.sum(axis=1)>1]

In [157]:
indexs

,,street,No.
419,5076,1.0,1
4429,17210,1.0,1
3327,5181,1.0,1
10552,10626,1.0,1
2671,18064,1.0,1
4685,11853,1.0,1
9203,15424,1.0,1
7304,18214,1.0,1
2655,17171,1.0,1
8672,14448,1.0,1


# process property assessment

In [176]:
property_assess = pd.read_csv('montreal-property-assessment.csv',encoding='utf-8')

In [181]:
property_assess = property_assess[['full_address','borough','neighbourhood_unit_number','taxable_value_of_building','retrieved_at']]

In [237]:
map_muncipality_code = dict()
for i in collections.Counter(property_assess.borough).keys()[1:]:
    
    print i
    
    dis = pd.Series()
    for j in muncipality_name:
        dis.set_value(j, 
                      jellyfish.jaro_winkler(i.replace('Arrondissement de ','').replace("Arrondissement d'",'')
                                             .replace("Arrondissement du ",''),muncipality_name[j]))
        
    print np.argmax(dis),dis.max(),'\n'
    
    map_muncipality_code[i] = np.argmax(dis)

Arrondissement de LaSalle
820 0.8777777777777778 

Arrondissement de Lachine
823 0.8777777777777778 

Arrondissement de Ville-Marie
843 0.9 

Arrondissement de Côte-des-Neiges - Notre-Dame-de-Grâce
838 0.8842202450898103 

Arrondissement de Pierrefonds - Roxboro
835 0.8838095238095239 

Arrondissement de Saint-Léonard
816 0.9083333333333333 

Arrondissement d'Outremont
825 0.89 

Arrondissement du Plateau-Mont-Royal
842 0.8171296296296297 

Arrondissement de Montréal-Nord
817 0.9083333333333333 

Arrondissement de Rivière-des-Prairies - Pointe-aux-Trembles
846 0.8973389355742297 

Arrondissement de Verdun
819 0.8375 

Arrondissement de Rosemont - La Petite-Patrie
844 0.868074074074074 

Arrondissement de Saint-Laurent


/Users/lingao/.virtualenvs/scratch/lib/python2.7/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


827 0.9083333333333333 

Arrondissement d'Anjou
815 0.8625 

Arrondissement d'Ahuntsic-Cartierville
840 0.93125 

Arrondissement du Sud-Ouest
839 0.6859903381642511 

Arrondissement de Villeray - Saint-Michel - Parc-Extension
841 0.8496504559270517 

Arrondissement de L'Île-Bizard - Sainte-Geneviève
837 0.876924360400445 

Arrondissement de Mercier - Hochelaga-Maisonneuve
845 0.8849308755760369 



In [250]:
muncipality_name

{814: u'Montr\xe9al-Est',
 815: u'Anjou (Montr\xe9al)',
 816: u'Saint-L\xe9onard (Montr\xe9al)',
 817: u'Montr\xe9al-Nord (Montr\xe9al)',
 818: u'Westmount',
 819: u'Verdun/\xcele-des-Soeurs (Montr\xe9al)',
 820: u'LaSalle (Montr\xe9al)',
 821: u'Montr\xe9al-Ouest',
 822: u'C\xf4te-Saint-Luc',
 823: u'Lachine (Montr\xe9al)',
 824: u'Hampstead',
 825: u'Outremont (Montr\xe9al)',
 826: u'Mont-Royal',
 827: u'Saint-Laurent (Montr\xe9al)',
 828: u'Dorval',
 829: u'Pointe-Claire',
 830: u'Kirkland',
 831: u'Beaconsfield',
 832: u"Baie-d'Urf\xe9",
 833: u'Sainte-Anne-de-Bellevue',
 834: u'Senneville',
 835: u'Pierrefonds-Roxboro (Montr\xe9al)',
 836: u'Dollard-Des Ormeaux',
 837: u"L'\xcele-Bizard/Sainte-Genevi\xe8ve (Montr\xe9al)",
 838: u'C\xf4te-des-Neiges/Notre-Dame-de-Gr\xe2ce (Montr\xe9al)',
 839: u'Le Sud-Ouest (Montr\xe9al)',
 840: u'Ahuntsic-Cartierville (Montr\xe9al)',
 841: u'Villeray/Saint-Michel/Parc-Extension (Montr\xe9al)',
 842: u'Le Plateau-Mont-Royal (Montr\xe9al)',
 843: u

In [251]:
map_muncipality_code

{u"Arrondissement d'Ahuntsic-Cartierville": 840,
 u"Arrondissement d'Anjou": 815,
 u"Arrondissement d'Outremont": 825,
 u'Arrondissement de C\xf4te-des-Neiges - Notre-Dame-de-Gr\xe2ce': 838,
 u"Arrondissement de L'\xcele-Bizard - Sainte-Genevi\xe8ve": 837,
 u'Arrondissement de LaSalle': 820,
 u'Arrondissement de Lachine': 823,
 u'Arrondissement de Mercier - Hochelaga-Maisonneuve': 845,
 u'Arrondissement de Montr\xe9al-Nord': 817,
 u'Arrondissement de Pierrefonds - Roxboro': 835,
 u'Arrondissement de Rivi\xe8re-des-Prairies - Pointe-aux-Trembles': 846,
 u'Arrondissement de Rosemont - La Petite-Patrie': 844,
 u'Arrondissement de Saint-Laurent': 827,
 u'Arrondissement de Saint-L\xe9onard': 816,
 u'Arrondissement de Verdun': 819,
 u'Arrondissement de Ville-Marie': 843,
 u'Arrondissement de Villeray - Saint-Michel - Parc-Extension': 841,
 u'Arrondissement du Plateau-Mont-Royal': 842,
 u'Arrondissement du Sud-Ouest': 839}

In [242]:
#property_assess['muncipality_id'] = 
muncipality_id = [np.nan] * len(property_assess)
for i in range(len(property_assess)):
    try:
        muncipality_id[i] = map_muncipality_code[property_assess.iloc[i,1]]
    except:
        pass

In [245]:
property_assess['muncipality_id'] = muncipality_id

/Users/lingao/.virtualenvs/scratch/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [254]:
property_assess

,full_address,borough,neighbourhood_unit_number,taxable_value_of_building,retrieved_at,muncipality_id
0,6937 - 6951 Avenue Mousseau,Arrondissement d'Anjou,709.0,315200.0,2018-07-11,815.0
1,"Boulevard Henri-Bourassa Est, LOT ARR.",Arrondissement de Rivière-des-Prairies - Point...,1937.0,483700.0,2018-07-11,846.0
2,9229 Rue Pierre-Bonne,Arrondissement de Rivière-des-Prairies - Point...,1928.0,5500000.0,2018-07-11,846.0
3,4330 - 4336 Rue de Verdun,Arrondissement de Verdun,1088.0,402500.0,2018-07-11,819.0
4,438 - 440 Rue Saint-Pierre,Arrondissement de Ville-Marie,2042.0,2945700.0,2018-07-11,843.0
5,"Rue Jean-Talon Est, LOT ARR",Arrondissement de Saint-Léonard,1276.0,901000.0,2018-07-11,816.0
6,901 Rue Maheu,Arrondissement de Saint-Laurent,1354.0,606500.0,2018-07-11,827.0
7,2575 Rue Remembrance,Arrondissement de Lachine,3651.0,613000.0,2018-07-11,823.0
8,8201 Place Marien,NaN,1750.0,4800000.0,2018-07-11,NaN
9,7052 Croissant de la Berge,Arrondissement d'Anjou,719.0,570600.0,2018-07-11,815.0
